Menghitung nilai MASV menggunakan model Random Forest  

In [24]:
import shap
import numpy as np
import pandas as pd
import time 
import ast
from xgboost import XGBClassifier

In [ ]:
# 1. Load the saved dataset
dataset = pd.read_csv("results/cicids2017_cleaned_numericable.csv")

# Display the first 5 rows of the dataset
print("First 5 Rows of the Dataset:")
print(dataset.head())

# Display the class distribution in the 'Label' column
print("\nClass Distribution:")
print(dataset['Label'].value_counts())


First 5 Rows of the Dataset:
   Destination Port  Flow Duration  Total Fwd Packets  Total Backward Packets  \
0                53          62564                  2                       2   
1                53            222                  2                       2   
2             62194            331                  3                       1   
3               443         423498                 14                      10   
4                80      115826152                 18                      18   

   Total Length of Fwd Packets  Total Length of Bwd Packets  \
0                           70                          228   
1                           84                          168   
2                           43                            6   
3                          841                         5082   
4                         1893                         7141   

   Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
0                     35      

In [26]:
# 2. Separate features and labels (assuming the last column is the label)
X = dataset.drop(columns=['Label'])
y = dataset['Label']

In [27]:
# 3. Train the XGBoost model
xgb_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X, y)

`use_label_encoder` is deprecated in 1.7.0.


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob',
              predictor=None, ...)

In [28]:
# 4. Compute SHAP values for the XGBoost model
# Using TreeExplainer without a subset
explainer = shap.TreeExplainer(xgb_model)
start_time = time.time()
shap_values = explainer.shap_values(X)
elapsed_time = time.time() - start_time
print(f"\n⏱️ SHAP computation time: {elapsed_time:.2f} seconds")

ntree_limit is deprecated, use `iteration_range` or model slicing instead.



⏱️ SHAP computation time: 591.77 seconds


In [ ]:
def mean_abs_shap_values(shap_values, X):
    """
    Computes the mean absolute SHAP values for multiclass or binary classification.
    """
    print(f"Shape of SHAP values: {np.shape(shap_values)}")  # Debugging: display the shape of shap_values
    print(f"Shape of X: {X.shape}")  # Debugging: display the shape of X

    # Multiclass handling: SHAP values are a list (one array per class)
    if isinstance(shap_values, list):  # Multiclass: shap_values is a list
        print("Multiclass case detected.")  # Debugging
        # Take the absolute SHAP values and compute the mean across samples and classes
        shap_abs_values = np.mean([np.abs(class_shap) for class_shap in shap_values], axis=0)  # Mean across classes
    elif shap_values.ndim == 3:  # Multiclass with 3D array (n_samples, n_features, n_classes)
        print("Multiclass array case detected.")  # Debugging
        shap_abs_values = np.abs(shap_values).mean(axis=0).mean(axis=1)  # Mean over samples and classes
    else:
        # Binary or regression case
        print("Binary/Regression case detected.")  # Debugging
        shap_abs_values = np.abs(shap_values).mean(axis=0)  # Take the absolute value and mean across samples

    print(f"Shape of shap_abs_values after processing: {shap_abs_values.shape}")  # Debugging

    # Ensure the number of features matches the number of columns in X
    num_features = X.shape[1]
    if shap_abs_values.shape[0] != num_features:
        raise ValueError(f"Shape of SHAP values {shap_abs_values.shape} does not match the number of features {num_features}.")

    # Create a DataFrame from the mean absolute SHAP values
    shap_importance = pd.DataFrame(shap_abs_values, index=X.columns, columns=['Mean |SHAP|'])

    # Ensure the order matches the order of columns in X
    shap_importance = shap_importance.loc[X.columns]
    
    return shap_importance

# ========== MODIFIKASI: Catat waktu proses ==========
start_time = time.time()
# Panggil fungsi dengan perbaikan
try:
    shap_importance_xgb = mean_abs_shap_values(shap_values, X)

    # Menampilkan beberapa baris pertama hasil
    print(shap_importance_xgb.head())  # Menampilkan 5 fitur teratas dengan MASV tertinggi

    # Menyimpan hasil ke CSV tanpa urutan ranking
    shap_importance_xgb.to_csv("results/shap_xgb_no_ranking.csv")

    print("Mean absolute SHAP values untuk XGBoost telah disimpan ke file CSV.")
except ValueError as e:
    print(f"Error: {e}")

end_time = time.time()
duration = end_time - start_time
print(f"\n⏱️ Waktu perhitungan MASV: {duration:.2f} detik")

Shape of SHAP values: (7, 1413949, 77)
Shape of X: (1413949, 77)
Multiclass case detected.
Shape of shap_abs_values after processing: (1413949, 77)
Error: Shape of SHAP values (1413949, 77) does not match the number of features 77.

⏱️ Waktu perhitungan MASV: 3.63 detik


In [ ]:
# Read the CSV file
df = pd.read_csv("results/shap_xgb_no_ranking.csv")

# Set display option to show all rows
pd.set_option('display.max_rows', None)  # Display all rows

# Display the DataFrame after loading
print(df)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/user/Desktop/shap-based_fs_smote/results/shap_xgb_no_ranking.csv'

In [ ]:
# Rename columns
df.rename(columns={'Unnamed: 0': 'feature', 'Mean |SHAP|': 'masv'}, inplace=True)

# Set display option to show all rows
pd.set_option('display.max_rows', None)  # Display all rows

# Display the DataFrame after renaming columns
print(df)

                        feature      masv
0              Destination Port  1.658366
1                 Flow Duration  0.133395
2             Total Fwd Packets  0.066365
3        Total Backward Packets  0.022520
4   Total Length of Fwd Packets  0.193250
5   Total Length of Bwd Packets  0.049359
6         Fwd Packet Length Max  0.088659
7         Fwd Packet Length Min  0.023400
8        Fwd Packet Length Mean  0.057904
9         Fwd Packet Length Std  0.133472
10        Bwd Packet Length Max  0.071929
11        Bwd Packet Length Min  0.073077
12       Bwd Packet Length Mean  0.146394
13        Bwd Packet Length Std  0.189442
14                 Flow Bytes/s  0.246137
15               Flow Packets/s  0.058912
16                Flow IAT Mean  0.093172
17                 Flow IAT Std  0.126996
18                 Flow IAT Max  0.156475
19                 Flow IAT Min  0.265135
20                Fwd IAT Total  0.115806
21                 Fwd IAT Mean  0.058377
22                  Fwd IAT Std  0

In [ ]:
num_rows = len(df)
print("Number of rows in the DataFrame:", num_rows)

Jumlah baris dalam DataFrame: 77


In [ ]:
# Save back to CSV file
df.to_csv("results/xgbmasv.csv", index=False)